# Abbreviation matching

* [scispacy abbreviationdetector](https://github.com/allenai/scispacy#abbreviationdetector) can register the provisioned abbreviations e.g. ```StackOverflow (SO)``` and record their locations.

In [3]:
import re
import spacy
from scispacy.abbreviation import (
    AbbreviationDetector
)

# Sci Spacy Language Model

In [9]:
#spacy.download('en_core_sci_lg')
nlp = spacy.load("en_core_sci_lg")

/home/eml/venv/ml/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_lg' (0.5.3) was trained with spaCy v3.6.1 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/eml/venv/ml/lib/python3.10/site-packages/spacy/language.py:2170: FutureWarning: Possible set union at position 6328
  serializers["tokenizer"] = lambda: self.tokenizer.to_bytes(exclude=["vocab"])  # type: ignore[union-attr]


## Abbreviation Detector into the pipeline

In [10]:
nlp.add_pipe("abbreviation_detector")

# Register the abbreviations and detection

In [11]:
text = """
Spinal and bulbar muscular atrophy (SBMA) is an inherited motor neuron disease caused by the expansion
of a polyglutamine tract within the androgen receptor (AR). SBMA can be caused by this easily.

Active assisted range of movement (AAROM) gives assistance from an outside force. 
This range of motion may result from the majority of motion applied by an exerciser or 
by the person or persons assisting the individual. 
It also may be a half-and-half effort on the joint from each source.
"""
doc = nlp(' '.join(text.split()))

# Detected abbreviations

Word positions ```(token.i)``` in the text.

In [12]:
for abrv in doc._.abbreviations:
    print(f"{abrv.text:<15}: ({abrv.start}, {abrv.end}) {abrv._.long_form}")
    
abbreviations = {
    abrv.start: abrv._.long_form for abrv in doc._.abbreviations
}

SBMA           : (30, 31) Spinal and bulbar muscular atrophy
SBMA           : (6, 7) Spinal and bulbar muscular atrophy
AR             : (27, 28) androgen receptor
AAROM          : (44, 45) Active assisted range of movement


In [13]:
for token in doc:
    print(f"{token.i:<4} {token.text:20} : {abbreviations.get(token.i, '')}")

0    Spinal               : 
1    and                  : 
2    bulbar               : 
3    muscular             : 
4    atrophy              : 
5    (                    : 
6    SBMA                 : Spinal and bulbar muscular atrophy
7    )                    : 
8    is                   : 
9    an                   : 
10   inherited            : 
11   motor                : 
12   neuron               : 
13   disease              : 
14   caused               : 
15   by                   : 
16   the                  : 
17   expansion            : 
18   of                   : 
19   a                    : 
20   polyglutamine        : 
21   tract                : 
22   within               : 
23   the                  : 
24   androgen             : 
25   receptor             : 
26   (                    : 
27   AR                   : androgen receptor
28   )                    : 
29   .                    : 
30   SBMA                 : Spinal and bulbar muscular atrophy
31   can        

In [14]:
token = doc[0]
token.i

0

In [18]:
def replace_abbreviation(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]
    for abrv in doc._.abbreviations:
        tokens[abrv.start] = str(abrv._.long_form)
        
    return(" ".join(tokens))

print(replace_abbreviation(text))


 Spinal and bulbar muscular atrophy ( Spinal and bulbar muscular atrophy ) is an inherited motor neuron disease caused by the expansion 
 of a polyglutamine tract within the androgen receptor ( androgen receptor ) . Spinal and bulbar muscular atrophy can be caused by this easily . 

 Active assisted range of movement ( Active assisted range of movement ) gives assistance from an outside force . 
 This range of motion may result from the majority of motion applied by an exerciser or 
 by the person or persons assisting the individual . 
 It also may be a half-and-half effort on the joint from each source . 

